In [ ]:
import psycopg2

import pandas as pd
import numpy as np

!pip install pycountry
import pycountry

## Postgres Connection

In [ ]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    port="5432",
    database="postgres",
    user="ag_class",
    password="WUcgdfQ1")

conn

<connection object at 0x7f7edd328040; dsn: 'user=ag_class password=xxx dbname=postgres host=awesome-hw.sdsc.edu port=5432', closed: 0>

In [ ]:
cursor = conn.cursor()

## Extracting Data from PostgreSQL Patent DB

In [ ]:
query1 = '''
SELECT * FROM postgres.public.patentdb 
'''

data_df = pd.read_sql(query1, conn)

In [ ]:
data_df.columns

Index(['patentid', 'application_number', 'inventor_name',
       'assignee_name_origin', 'assignee_name_current', 'time_events', 'cite',
       'classification', 'classname', 'countrycode', 'abstract', 'title', 'id',
       'issuing_country'],
      dtype='object')

## Function to get industries

In [ ]:
def extract_industry(row):
    class_arr = row["classname"]
    if len(class_arr) == 0:
        return None
    else : 
        class_arr1 = [ele for ele in class_arr if len(ele.split())<5]
        class_set = set(class_arr1)
        class_set = {y for x in class_set for y in x.split('; ')}
        class_arr2 = list(class_set)
        class_arr2.sort(key=lambda x: len(x.split()))
        class_arr2 = class_arr2[:10]
        return class_arr2

## Function to get files and granted date

In [ ]:
def extract_date(row):
    class_arr = row["time_events"]
    if len(class_arr) == 0:
        return None,None
    else : 
      try:
        temp1 = [d['date'] for d in class_arr if d['title']=='filed']
        temp2 = [d['date'] for d in class_arr if d['title']=='granted']
        return temp1[0],temp2[0]
      except:
        return None,None

## Function to get Country Name from Country Code

In [ ]:
def country_name(i):
  
  try: 
    if len(i)==3:
      return pycountry.countries.get(alpha_3=i).name
    elif len(i)==2:
      return pycountry.countries.get(alpha_2=i).name
    else:
      return None
  except:
    if i == 'EP':
      return 'European Patent Office'
    if i== 'WO':
      return 'World Intellectual Property Organisation'
    if i== 'EA':
      return 'Eurasian Patent Organisation'
    if i== 'CS':
      return 'Czech Republic and Slovakia'
    if i== 'SU':
      return 'Soviet Union' 
    if i== 'DD':
      return 'German Democratic Republic'
    if i== 'OA':
      return 'African Intellectual Property Organization'
    if i== 'AP':
      return 'Asia/Pacific Region' 
    if i== 'YU':
      return 'Yugoslavia' 
    else:
      print(i)
      return None

## Function to get Country Name from Country Code

In [ ]:
def get_countrycode(i):

  if i== 'European Patent Office (EPO)':
      return 'EP'
  if i== 'Taiwan Province of China':
      return 'TW'
  if i== 'Republic of Korea':
      return 'KR'
  if i== 'Eurasian Patent Organization (EAPO)':
      return 'EA'
  if i== 'African Intellectual Property Organization (OAPI)':
      return 'OA'
  if i== 'African Regional Intellectual Property Organization ':
      return 'AP'
  if i =='German Democratic Republic':
      return 'DD'
  if i =='World Intellectual Property Organization':
      return 'WO'
  if i =='USSR':
      return 'SU'
  if i =='Czechoslovakia':
      return 'CS'
  if i =='USSR':
      return 'SU'
  
  try:
    return pycountry.countries.get(official_name=i).alpha_2
  except:
    return pycountry.countries.get(name=i).alpha_2

## Creating CSVs for Nodes and Relationships

In [ ]:
data_df['assignee_name_current'] = np.where(data_df['assignee_name_current'].astype(str)[0] == '[]', data_df['assignee_name_origin'], data_df['assignee_name_current'])

In [ ]:
organisations = data_df['assignee_name_current'].explode()

mask_nonascii = organisations.map(lambda x: len(str(x).encode('ascii', errors='ignore')) != len(str(x)))

In [ ]:
organisations[~mask_nonascii].drop_duplicates().to_csv('Organizations.csv',index=False)

In [ ]:
organizations_new = organisations[~mask_nonascii].str.strip().drop_duplicates().replace(np.nan,'').sort_values(ascending=False).reset_index(drop=True)

In [ ]:
organizations_new = organizations_new.map(lambda x: x.lstrip('*').strip())

In [ ]:
organizations_new = organizations_new[organizations_new!=''].str.title()

In [ ]:
organizations_new.to_csv('Organizations.csv',index=False)

In [ ]:
data_df["clean_industry"] = data_df.apply(lambda row : extract_industry(row), axis=1)

In [ ]:
industry = data_df["clean_industry"].explode().sort_values()

In [ ]:
industry = industry[(industry!='') & (~industry.isnull())].drop_duplicates().str.strip().str.title()

In [ ]:
industry.to_csv('Industries.csv',index=False)

In [ ]:
patents = data_df[['patentid', 'inventor_name', 'time_events', 'countrycode', 'title','issuing_country']]

In [ ]:
patents['Filed Date'],patents['Granted Date'] = zip(*patents.apply(lambda row : extract_date(row), axis=1))

<ipython-input-20-ddfbc51994c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patents['Filed Date'],patents['Granted Date'] = zip(*patents.apply(lambda row : extract_date(row), axis=1))


In [ ]:
patents = patents.drop("time_events",axis=1).explode('inventor_name').explode('countrycode')

In [ ]:
patents = patents.drop_duplicates().dropna()

In [ ]:
mask_nonascii = patents['inventor_name'].map(lambda x: len(str(x).encode('ascii', errors='ignore')) != len(str(x)))

In [ ]:
patents = patents[~mask_nonascii].drop_duplicates().dropna()

In [ ]:
patents.columns

Index(['patentid', 'inventor_name', 'countrycode', 'title', 'issuing_country',
       'Filed Date', 'Granted Date'],
      dtype='object')

In [ ]:
patents = patents.groupby(['patentid', 'title', 'issuing_country','Filed Date', 'Granted Date'],as_index=False).agg({'inventor_name': 'unique','countrycode': 'unique'})

In [ ]:
patents['inventor_name'] = patents['inventor_name'].str.join(',')
patents['countrycode'] = patents['countrycode'].str.join(',')

In [ ]:
patents.to_csv('Patents.csv',index=False)

In [ ]:
patent_industry = data_df[['patentid','clean_industry']].explode('clean_industry')

In [ ]:
patent_industry = patent_industry[(patent_industry['clean_industry']!='') & (~patent_industry['clean_industry'].isnull())]

In [ ]:
patent_industry['clean_industry'] = patent_industry['clean_industry'].str.strip().str.title()

In [ ]:
patent_industry = patent_industry[patent_industry['patentid'].isin(patents['patentid'].unique())].drop_duplicates()

In [ ]:
patent_industry.to_csv('Patent_Industry.csv',index=False)

In [ ]:
patent_assignee = data_df[['patentid','assignee_name_current']].explode('assignee_name_current').drop_duplicates()

In [ ]:
mask_nonascii = patent_assignee['assignee_name_current'].map(lambda x: len(str(x).encode('ascii', errors='ignore')) != len(str(x)))

In [ ]:
patent_assignee = data_df[['patentid','assignee_name_current']].explode('assignee_name_current').drop_duplicates()
patent_assignee[~mask_nonascii]

,patentid,assignee_name_current
0,US7635039B2,Mitsubishi Heavy Industries Ltd
1,US4904344A,Valmet Technologies Oy
1,US4904344A,Mitsubishi Heavy Industries Ltd
2,US7982327B2,Mitsubishi Heavy Industries Ltd
3,US6607353B2,Mitsubishi Heavy Industries Ltd
...,...,...
156035,US8413929B2,Mitsubishi Heavy Industries Ltd
156036,US6818072B2,Mitsubishi Heavy Industries Ltd
156037,US6573493B1,Mitsubishi Heavy Industries Ltd
156038,US6338371B1,Mitsubishi Heavy Industries Ltd


In [ ]:
patent_assignee['assignee_name_current']=patent_assignee['assignee_name_current'].str.strip().replace(np.nan,'')

In [ ]:
patent_assignee['assignee_name_current'] = patent_assignee['assignee_name_current'].map(lambda x: x.lstrip('*').strip()).str.title()

In [ ]:
patent_assignee = patent_assignee[patent_assignee['assignee_name_current']!=''].drop_duplicates().reset_index(drop=True)

In [ ]:
patent_assignee.to_csv("Patent_Assignee.csv",index=False)

In [ ]:
patent_country = data_df[['patentid','countrycode']].explode('countrycode').drop_duplicates().reset_index(drop=True)

In [ ]:
patent_country.shape

(350017, 2)

In [ ]:
patent_country = patent_country.dropna()

In [ ]:
country = patent_country['countrycode'].drop_duplicates().reset_index(drop=True)

In [ ]:
country = pd.DataFrame(country)

In [ ]:
country['countryname'] = country['countrycode'].apply(country_name)

In [ ]:
country.to_csv('Countries.csv',index=False)

In [ ]:
patent_country = data_df[['patentid','issuing_country']].dropna().drop_duplicates().reset_index(drop=True)
patent_country.to_csv('Patent_IssuingCountry.csv',index=False)

In [ ]:
issuingcountry = pd.DataFrame(patent_country['issuing_country'].drop_duplicates().reset_index(drop=True))

In [ ]:
patent_country

,patentid,issuing_country
0,US7635039B2,United States of America
1,US4904344A,United States of America
2,US7982327B2,United States of America
3,US6607353B2,United States of America
4,US7030354B2,United States of America
...,...,...
78886,US8413929B2,United States of America
78887,US6818072B2,United States of America
78888,US6573493B1,United States of America
78889,US6338371B1,United States of America


In [ ]:
pycountry.countries.get(alpha_3='USA')

Country(alpha_2='US', alpha_3='USA', flag='🇺🇸', name='United States', numeric='840', official_name='United States of America')

In [ ]:
issuingcountry['countrycode'] = issuingcountry['issuing_country'].apply(get_countryname)

In [ ]:
patent_country['countrycode'] = patent_country['issuing_country'].apply(get_countryname)

In [ ]:
issuingcountry[['countrycode','issuing_country']].to_csv('IssuingCountryCode.csv',index=False)

In [ ]:
patent_country[['patentid','countrycode']].to_csv('PatentIssuingCouuntryCode.csv',index=False)